In [1]:
from ntscraper import Nitter
import requests
import json
import random

# URL da sua API
login_url = 'http://localhost:3000/auth/login'
url = 'http://localhost:3000/usuarios'

# Credenciais do usuário
credentials = {
    'usuario': 'admin',
    'senha': 'admin'
}

# Fazendo uma solicitação POST para obter o token JWT
login_response = requests.post(login_url, json=credentials)

if login_response.status_code == 200:
    token = login_response.json()['token']
    
    # Incluindo o token JWT no cabeçalho Authorization
    headers = {
        'Authorization': "Bearer " + token
    }

    # Fazendo uma solicitação GET para a API protegida
    response = requests.get(url, headers=headers)

    # Verificando se a solicitação foi bem-sucedida (código de status 200)
    if response.status_code == 200:
        # Se a solicitação foi bem-sucedida, você pode acessar os dados da resposta assim:
        data = response.json()  # Isso assume que sua API retorna dados no formato JSON
        
        # Inicializa listas vazias para armazenar os nomes de perfil, ids de usuário e nomes de perfil
        perfis = []
        ids_usuarios = []
        nomes_perfis = []

        # Percorre a resposta da API
        for usuario in data:
            # Itera sobre os perfis do usuário
            for perfil in usuario.get('perfis', []):
                ids_usuarios.append(usuario['_id'])
                nomes_perfis.append(perfil['usuario'])
                perfis.append(perfil['usuario'])

        # Imprime as listas
        print("IDs de Usuários:", ids_usuarios)
        print("Nomes de Perfis:", nomes_perfis)
        print("Perfis:", perfis)
    else:
        print("Erro ao acessar a API. Código de status:", response.status_code)
else:
    print("Erro ao fazer login. Código de status:", login_response.status_code)

IDs de Usuários: ['66772a2c3e11c4f746ec936b', '66772a2c3e11c4f746ec936b', '66772a2c3e11c4f746ec936b', '66772a2c3e11c4f746ec936b', '66772a2c3e11c4f746ec936b']
Nomes de Perfis: ['Germancanoofi', 'jhonariasa', 'fredgol9', 'phganso', 'tsilva3']
Perfis: ['Germancanoofi', 'jhonariasa', 'fredgol9', 'phganso', 'tsilva3']


In [2]:
from pprint import pprint
import requests
import json
import random
import ntscraper

# Lista de instâncias Nitter
instances = [
    "https://nitter.lucabased.xyz",
    "https://nitter.privacydev.net"
]

# Função para tentar buscar tweets com uma instância
def get_tweets_with_instance(scraper, username, number, instances):
    for _ in range(len(instances)):
        instance = random.choice(instances)
        try:
            tweets = scraper.get_tweets(username, mode="user", number=number, instance=instance)
            if tweets and tweets.get('tweets'):
                return tweets
            else:
                print(f"Nenhum tweet encontrado usando a instância {instance}")
                instances.remove(instance)
        except Exception as e:
            print(f"Erro ao usar a instância {instance}: {e}")
            instances.remove(instance)
    raise Exception("Não foi possível obter tweets de nenhuma instância.")

# Inicializa o scraper
scraper = ntscraper.Nitter()

# Percorre os perfis, ids_usuarios e nomes_perfis
for usuarioId, nomePerfil in zip(ids_usuarios, nomes_perfis):
    # Busca tweets usando uma instância aleatória
    try:
        tweets = get_tweets_with_instance(scraper, nomePerfil, 1, instances.copy())
    except Exception as e:
        print(f"Erro ao obter tweets para o perfil {nomePerfil}: {e}")
        continue
    
    # Inicializa uma lista para armazenar os tweets processados deste perfil
    tweets_processados = []
    
    # Itera sobre cada tweet no dicionário de tweets
    for tweet in tweets['tweets']:
        # Cria um novo dicionário com os campos desejados
        tweet_processado = {
            'link': tweet['link'],
            'texto': tweet['text'],
            'usuarioId': usuarioId,  # Inclui o usuarioId aqui
            'nomePerfil': nomePerfil  # Inclui o nomePerfil aqui
        }
        # Adiciona o tweet processado à lista de tweets processados deste perfil
        tweets_processados.append(tweet_processado)

    # Exibe os tweets processados deste perfil
    for tweet in tweets_processados:
        pprint(tweet)

    # URL da sua API para onde deseja fazer o POST
    url2 = 'http://localhost:3000/tweets'

    # Cabeçalhos para indicar que o conteúdo é JSON
    headers = {'Content-Type': 'application/json', 'Authorization': "Bearer " + token}

    # Fazendo a requisição POST para cada tweet individualmente
    for tweet in tweets_processados:
        # Dados que serão enviados no corpo da requisição POST
        dados = json.dumps(tweet)

        # Fazendo a requisição POST
        response = requests.post(url2, data=dados, headers=headers)

        # Verificando se a requisição foi bem-sucedida
        if response.status_code == 201:
            print("Tweet inserido com sucesso!")
            
            # Extrai o ID do tweet inserido diretamente da resposta
            id_inserido = response.json()['tweet']['_id']

            # Imprime o ID do tweet inserido
            print("ID do tweet inserido:", id_inserido)
        else:
            print("Erro ao inserir tweet:", response.text)

Testing instances:  92%|██████████████████████████████████████████████████████████     | 71/77 [02:22<00:09,  1.62s/it]

23-Jun-24 17:00:24 - Certificate did not match expected hostname: nt.ggtyler.dev. Certificate: {'subject': ((('commonName', '4g.ggtyler.dev'),),), 'issuer': ((('countryName', 'US'),), (('organizationName', "Let's Encrypt"),), (('commonName', 'R3'),)), 'version': 3, 'serialNumber': '03A4459CE6DA31CFF555DB78FF3F35BFEA88', 'notBefore': 'May 14 10:55:44 2024 GMT', 'notAfter': 'Aug 12 10:55:43 2024 GMT', 'subjectAltName': (('DNS', '4g.ggtyler.dev'),), 'OCSP': ('http://r3.o.lencr.org',), 'caIssuers': ('http://r3.i.lencr.org/',)}


Testing instances:  94%|██████████████████████████████████████████████████████████▉    | 72/77 [02:22<00:06,  1.38s/it]

23-Jun-24 17:00:25 - Certificate did not match expected hostname: nitter.uni-sonia.com. Certificate: {'subject': ((('commonName', '*.xserver.jp'),),), 'issuer': ((('countryName', 'JP'),), (('organizationName', 'CloudSecure Corporation'),), (('commonName', 'CloudSecure RSA Domain Validation Secure Server CA 2'),)), 'version': 3, 'serialNumber': 'ACA67AD2030638EE2DCE8E845B8299A6', 'notBefore': 'Mar 11 00:00:00 2024 GMT', 'notAfter': 'Apr 11 23:59:59 2025 GMT', 'subjectAltName': (('DNS', '*.xserver.jp'), ('DNS', 'xserver.jp')), 'OCSP': ('http://ocsp.sectigo.com',), 'caIssuers': ('http://crt.sectigo.com/CloudSecureRSADomainValidationSecureServerCA2.crt',)}


Testing instances: 100%|███████████████████████████████████████████████████████████████| 77/77 [02:38<00:00,  2.06s/it]


23-Jun-24 17:00:47 - Current stats for Germancanoofi: 1 tweets, 0 threads...
{'link': 'https://twitter.com/Germancanoofi/status/1763454223017001127#m',
 'nomePerfil': 'Germancanoofi',
 'texto': 'Otra vez en RÍO #RECOPA2024',
 'usuarioId': '66772a2c3e11c4f746ec936b'}
Tweet inserido com sucesso!
ID do tweet inserido: 66787eefcf15195e34aeb80d
Erro ao usar a instância https://nitter.privacydev.net: list index out of range
23-Jun-24 17:01:06 - Fetching error: Instance has been rate limited.Use another instance or try again later.
Erro ao usar a instância https://nitter.lucabased.xyz: Invalid instance
Erro ao obter tweets para o perfil jhonariasa: Não foi possível obter tweets de nenhuma instância.
23-Jun-24 17:01:09 - Fetching error: Instance has been rate limited.Use another instance or try again later.
Nenhum tweet encontrado usando a instância https://nitter.lucabased.xyz
Erro ao usar a instância https://nitter.privacydev.net: list index out of range
Erro ao obter tweets para o perfil fr